Problem statement : https://drive.google.com/file/d/1eVUC3bp3aC5V7IVW3vMDCUkirSnb1h_9/view?usp=sharing

Copyrights: U Dinesh Kumar. Business Analytics: The Science of Data-Driven Decision Making (p. 528). Wiley India. Kindle Edition.

<b>Decision Variables</b> 

X1 = Quantity (in kilogram) of DH to be produced  
X2 = Quantity (in kilogram) of TH to be produced 


<b> Objective function </b>

The objective is to maximize the profit. The profit on DH per kg is 45 and the profit  on TH per kg is 50. 

Maximize => 45 X1 + 50 X2 

<b> Constraints </b>

Constraint for corn flour => 20,000 grams of corn flour is available. Each kg of  AH requires 500 grams of corn flour and each kg of TH 500 grams of corn flour.  

Thus, the corresponding constraint is  500 X1 + 500 X2 <= 20000  

Constraint for sugar => 42,000 kg of sugar is available. Each kg of DH requires  750 grams of sugar and each kg of TH 625 grams of sugar. 

Thus the corresponding  constraint is  750 X1 + 625 X2 <= 42000  

Constraint for fruit and nut => 10,400 kg of fruit and nut is available. Each kg of  DH requires 150 grams of fruit and nut and each kg of TH requires 100 grams of fruit  and nut. 

Thus the corresponding constraint is  150 X1 + 100 X2 <= 10400  

Constraint for ghee => 9,600 kg of ghee is available. Each kg of DH requires 200  grams of ghee and each kg of TH requires 300 grams of ghee. 

Thus the corresponding  constraint is  200X1 + 300X2 <= 9600  

Maximum demand constraint => The maximum daily demand for DH and TH  are 50 kg and 20 kg, respectively, which can be written as 

X1 ≤ 50 and X2 ≤ 20. 

Implicit constraints => X1, X2 - Non-negative. 

X1 >= 0 and X2 >= 0


In [1]:
import numpy as np
from gurobipy import *

In [2]:
var_names = ['x1', 'x2']
res_names = ['cornflour', 'sugar', 'fruit_nut','ghee','x1','x2']
profit = np.array([45, 50])                      # unit profits
res_used = np.array([[500, 500], [750, 625], [150, 100], [200,300], [1,0], [0,1]])    # resource consumption
res_avail = np.array([20000, 42000, 10400, 9600, 50, 20])            # resource availability

n = len(profit)                                    # number of variables
m = len(res_avail)                                 # number of constraints
assert (res_used.shape == (m, n)) and (len(var_names) == n) and (len(res_names) == m)

In [3]:
model = Model()
x = model.addVars(n, name=var_names) # production variables
obj = model.setObjective(quicksum(profit[j] * x[j] for j in range(n)), GRB.MAXIMIZE)
con = []
for i in range(m):
    con.append(model.addConstr(quicksum(res_used[i, j] * x[j] for j in range(n)) <= res_avail[i], name=res_names[i]))
model.optimize()

Using license file C:\Users\Vinu\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 6 rows, 2 columns and 10 nonzeros
Model fingerprint: 0x5718e378
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [5e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+04]
Presolve removed 3 rows and 0 columns
Presolve time: 0.15s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+03   7.900000e+02   0.000000e+00      0s
       2    1.8800000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.24 seconds
Optimal objective  1.880000000e+03


In [4]:
print('Sensitivity Analysis (SA)\nObjVal =', model.ObjVal)
model.printAttr(['X', 'Obj', 'SAObjLow', 'SAObjUp'])
model.printAttr(['X', 'RC', 'LB', 'SALBLow', 'SALBUp', 'UB', 'SAUBLow', 'SAUBUp'])
model.printAttr(['Sense', 'Slack', 'Pi', 'RHS', 'SARHSLow', 'SARHSUp']) # Pi = shadow price = dual variable value
# NOTE: printAttr prints only rows with at least one NON-ZERO value, e.g. model.printAttr('X') prints only non-zero variable values

Sensitivity Analysis (SA)
ObjVal = 1880.0

    Variable            X          Obj     SAObjLow      SAObjUp 
----------------------------------------------------------------
          x1           24           45      33.3333           50 
          x2           16           50           45         67.5 

    Variable            X           RC           LB      SALBLow       SALBUp           UB      SAUBLow       SAUBUp 
--------------------------------------------------------------------------------------------------------------------
          x1           24            0            0         -inf           24          inf           24          inf 
          x2           16            0            0         -inf           16          inf           16          inf 

  Constraint        Sense        Slack           Pi          RHS     SARHSLow      SARHSUp 
------------------------------------------------------------------------------------------
   cornflour            <            0

In [5]:
print("X =", model.X, "\nObjVal =", model.ObjVal, "\nPi =", model.Pi) # reading model attributes
print("x0 =", x[0].X) # reading variables attributes
print("Pi0 =", con[0].Pi) # reading constraint attributes

X = [24.0, 16.0] 
ObjVal = 1880.0 
Pi = [0.07, 0.0, 0.0, 0.05, 0.0, 0.0]
x0 = 24.0
Pi0 = 0.07


In [6]:
help(Model.addVar)
#help(Model.addVars)
#help(Model.setObjective)
#help(Model.addConstr)
#help(Model.addConstrs)


    ROUTINE:
      addVar(lb, ub, obj, vtype, name, column)

    PURPOSE:
      Add a variable to the model.

    ARGUMENTS:
      lb (float): Lower bound (default is zero)
      ub (float): Upper bound (default is infinite)
      obj (float): Objective coefficient (default is zero)
      vtype (string): Variable type (default is GRB.CONTINUOUS)
      name (string): Variable name (default is no name)
      column (Column): Initial coefficients for column (default is None)

    RETURN VALUE:
      The created Var object.

    EXAMPLE:
      v = model.addVar(ub=2.0, name="NewVar")
    
